In [6]:
import nltk

gutenberg = nltk.corpus.gutenberg
corpus = [word.lower() for word in gutenberg.words()]

spl = 95*len(corpus)/100
train = corpus[:spl]
test = corpus[spl:]

# Remove rare words from the corpus
fdist = nltk.FreqDist(w for w in train)
vocabulary = set(map(lambda x: x[0], filter(lambda x: x[1] >= 5, fdist.iteritems())))

train = map(lambda x: x if x in vocabulary else "*unknown*", train)
test = map(lambda x: x if x in vocabulary else "*unknown*", test)

print train[1:10]
print test[1:10]

[u'emma', u'by', u'jane', '*unknown*', '*unknown*', u']', u'volume', u'i', u'chapter']
[u'from', u'the', '*unknown*', '*unknown*', u'.', u'long', u'i', u'was', '*unknown*']


In [3]:
a  = ["The", "Hello", "King"]
b = "The King is going mad"
for unk in a:
    b = b.replace(unk, "Bro")
print b
a.replace("The", "SSup")
print a

Bro Bro is going mad


AttributeError: 'list' object has no attribute 'replace'

In [14]:
import numpy as np
import os
import io
import re
import nltk
import random
import math
import sys
from nltk.tokenize import word_tokenize
import prep_data

class N_gram():
    
    def __init__(self, n, train_data, test_data):
       
        train_data = ''.join(train_data)
        train_tokens = word_tokenize(train_data)

        test_data = ''.join(test_data)
        test_tokens = word_tokenize(test_data)

        self.train_tokens = train_tokens
        self.test_tokens= test_tokens
        self.n = n

        return
    
    def normalize_train_data(self):

        train_tokens = self.train_tokens
        freq = prep_data.get_freq_count(train_tokens)

        unks = []
        for word in freq:
            if freq[word] < 2:
                unks.append(word)
        print "Unknowns in train data :{}".format(len(unks))

        for unk in unks:
            for i in range(0,len(train_tokens)):
                if train_tokens[i] == unk:
                    train_tokens[i] = "UNK"

        self.freq = prep_data.get_freq_count(train_tokens)
        self.train_tokens = train_tokens

        # Random Printing
        firstkpairs = {k: freq[k] for k in freq.keys()[:10]}
        print firstkpairs


    def update_ngram_prob(self):

        train_tokens = self.train_tokens
        n =  self.n

        prob = dict()
        for i in range(0, len(train_tokens) - n + 1):
            ng = ' '.join(train_tokens[i:i+n])
            if ng not in prob:
                prob[ng] = 0
            prob[ng] += 1
        
        vocab_size = sum(self.freq.values())
        for element in prob:
            prob[element] = (prob[element] + 1.0)/(self.freq[element.split(' ')[0]] + vocab_size)

        self.prob = prob
        self.vocab_size =  vocab_size	
    
    def normalize_test_data(self):
       
        freq = self.freq
        unks = []

        for token in test_tokens:
            if token not in freq:
                unks.append(token)

        for unk in unks:
            for i in range(0,len(test_tokens)):
                if test_tokens[i] == unk:
                    test_tokens[i] = "UNK"

        self.test_tokens = test_tokens 


    def calc_perplexity(self):
        prob = self.prob
        vocab_size = self.vocab_size
        test_tokens = self.test_tokens
        n  = self.n
        freq =  self.freq

        test_new_prob = dict()
        total_log_prob = 0
 
        for i in range(0, len(test_tokens) - n + 1):
            ng = ' '.join(test_tokens[i:i+n])
            print ng
            if ng not in prob:
                if ng not in test_new_prob:
                    test_new_prob[ng] = math.log(1.0/(freq[ng.split(' ')[0]] + vocab_size))
                total_log_prob += test_new_prob[ng]
            else:
                total_log_prob += math.log(prob[ng])

        self.perplexity = (1.0/(total_log_prob))